In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Load Data ---
try:
    df = pd.read_csv('PCE.csv')
    if len(df.columns) == 1:
        df = pd.read_csv('PCE.csv', delimiter='\t')
except FileNotFoundError:
    print("Error: 'PCE.csv' not found.")
    exit()

# --- Preprocess ---
temp_cols = ['C400_1', 'C400_2', 'C400_3']
df['Mean_C400'] = df.get('Mean', df[temp_cols].mean(axis=1))
df['SEM_C400'] = df[temp_cols].sem(axis=1)
df.rename(columns={df.columns[0]: 'Time(s)'}, inplace=True)

# --- Cooling Phase ---
cooling_range = (600, 1200)
T_max = df['Mean_C400'].max()
T_surr = df.loc[df['Time(s)'] == 0, 'Mean_C400'].values[0]
print(f"T_max: {T_max:.2f} °C, T_surr: {T_surr:.2f} °C")

cool = df[(df['Time(s)'] >= cooling_range[0]) & (df['Time(s)'] <= cooling_range[1])].copy()
cool['Time_relative'] = cool['Time(s)'] - cooling_range[0]
cool['theta'] = (cool['Mean_C400'] - T_surr) / (T_max - T_surr)
cool['Neg_Ln_theta'] = -np.log(cool['theta'].mask(cool['theta'] <= 0.0001))

print("\nCooling Phase:\n", cool[['Time_relative', 'theta', 'Neg_Ln_theta']].to_string(index=False))

# --- Linear Fit ---
fit = cool.dropna(subset=['Neg_Ln_theta'])
if len(fit) >= 2:
    x_fit, y_fit = fit['Neg_Ln_theta'].values, fit['Time_relative'].values
    slope, intercept = np.polyfit(x_fit, y_fit, 1)
    y_pred = slope * x_fit + intercept
    r2 = 1 - np.sum((y_fit - y_pred)**2) / np.sum((y_fit - y_fit.mean())**2)
else:
    slope = intercept = r2 = 0

# --- Plot ---
fig, ax1 = plt.subplots(figsize=(8, 6))
pad = 0.05

# Temperature vs Time
ax1.errorbar(df['Time(s)'], df['Mean_C400'], yerr=df['SEM_C400'], fmt='-o',
             color='black', linewidth=2, capsize=2, markersize=8)
ax1.set_xlabel('Time (s)', fontsize=18, fontweight='bold', family='Arial')
ax1.set_ylabel('Temperature (°C)', fontsize=18, fontweight='bold', family='Arial')
ax1.set_xlim(0 - pad * df['Time(s)'].max(), df['Time(s)'].max() * (1 + pad))
y_min, y_max = df['Mean_C400'].min(), df['Mean_C400'].max()
ax1.set_ylim(y_min - pad * (y_max - y_min), y_max + pad * (y_max - y_min))
ax1.set_xticks(np.arange(0, df['Time(s)'].max() + 1, 200))
ax1.set_yticks(np.arange(np.floor(y_min / 5) * 5, np.ceil(y_max / 5) * 5 + 5, 5))
ax1.tick_params(length=12, width=3, labelsize=18)
ax1.spines['left'].set_linewidth(3)
ax1.spines['bottom'].set_linewidth(3)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

# Time_relative on right Y-axis
ax2 = ax1.twinx()
ax2.set_ylabel('Time (s)', fontsize=18, fontweight='bold', family='Arial', color='red')
ax2.set_ylim(-pad * 600, 600 + pad * 600)
ax2.tick_params(axis='y', colors='red', length=12, width=3, labelsize=18)
ax2.spines['right'].set_linewidth(3)
ax2.spines['right'].set_color('red')
ax2.spines['left'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['bottom'].set_visible(False)

# -ln(theta) on top X-axis
ax3 = ax2.twiny()
ax3.set_xlabel(r'$-\mathrm{Ln}(\theta)$', fontsize=18, fontweight='bold', family='Arial', color='red')
ax3.set_xlim(-pad * 5, 5 + pad * 5)
ax3.set_xticks(np.arange(0, 6, 1))
ax3.tick_params(axis='x', colors='red', length=12, width=3, labelsize=18)
ax3.spines['top'].set_linewidth(3)
ax3.spines['top'].set_color('red')
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.spines['right'].set_visible(False)

# Scatter and Fit Line
ax3.plot(cool['Neg_Ln_theta'], cool['Time_relative'], 'o', color='red', markersize=8)
if len(fit) >= 2:
    ax3.plot(x_fit, y_pred, color='red', linewidth=3)
    eq = f'y = {slope:.4f}x + {intercept:.2f}\n$R^2$ = {r2:.4f}'
    ax3.text(0.2, 0.1, eq, transform=ax3.transAxes, fontsize=16, color='red',
             verticalalignment='bottom', horizontalalignment='left')

plt.tight_layout()
plt.savefig('PCE.tif', dpi=600)
plt.show()
